In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Audio-text segment (Whisper + SBERT)

In [ ]:
# Install required packages
!pip install git+https://github.com/huggingface/transformers -q
!pip install torch torchaudio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Import necessary libraries
import torch
from transformers import pipeline

# Check available GPU
device = 0 if torch.cuda.is_available() else -1

# Initialize Whisper model for automatic speech recognition
whisper = pipeline('automatic-speech-recognition', model='openai/whisper-medium', device=device)

# Transcribe audio from the video using Whisper model
video_file_path = "video_en.mp4"  # Replace with your video file path
text = whisper(video_file_path)

# Print transcribed text
print("Transcribed Text:")
print(text)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcribed Text:
{'text': ' The expected rematch between President Biden and former President Donald Trump is happening. We\'re effectively in a general election now. That\'s just unprecedented in a lot of ways. There\'s just so many unknowns. While Biden faces obstacles like concerns about his age and declining approval ratings, Trump is juggling a set of challenges that no presidential candidates have ever confronted. Here\'s a look at those challenges and how Trump will try to turn them into opportunities. The 2024 election is gearing up to be a faceoff between the oldest nominees in history. And it\'s going to be the first presidential rematch in the U.S. since 1956, when President Dwight Eisenhower faced and defeated Adlai Stevenson for the second time. The result was never in doubt. It was a landslide for the Eisenhower-Nixon ticket. We know from polling and just, you know, talking to voters around the country that people are really kind of weary about the notion of a Trump-Bide

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize T5 model and tokenizer for text summarization
MODEL_NAME = 'cointegrated/rut5-base-absum'

# Check available GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Move model to GPU if available
t5_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def summarize_text_t5(text, max_length=100, num_beams=4, repetition_penalty=2.5):
    """
    Summarize the text using the T5 model.
    """
    # Encode text and move to GPU if available
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    with torch.inference_mode():
        summary_ids = t5_model.generate(input_ids, max_length=max_length, num_beams=num_beams,
                                        repetition_penalty=repetition_penalty)
    # Move summary to CPU for decoding
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def main():
    # Summarize the text using T5
    summarized_text_t5 = summarize_text_t5(text['text'])

    # Print the summarized text
    print("Summarized Text:")
    print(summarized_text_t5)

if __name__ == "__main__":
    main()


Summarized Text:
Republican President Donald Trump is expected to win the 2024 presidential election.


In [ ]:
# If the transcribed text is a dictionary and contains the 'text' key, extract the text
if isinstance(text, dict) and 'text' in text:
    transcribed_text = text['text']
else:
    transcribed_text = text

# import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize T5 model and tokenizer for text summarization
MODEL_NAME = "sarahai/ruT5-base-summarizer"
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

t5_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def summarize_text_t5(text, max_length=100, num_beams=4, repetition_penalty=2.5):
    """
    Summarize the text using the T5 model.
    """
    # Encode text and move to GPU if available
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    with torch.inference_mode():
        summary_ids = t5_model.generate(input_ids, max_length=max_length, num_beams=num_beams,
                                        repetition_penalty=repetition_penalty)
    # Move summary to CPU for decoding
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Summarize the transcribed text using T5
summarized_text_t5 = summarize_text_t5(transcribed_text)

# Print the summarized text
print("Summarized Text:")
print(summarized_text_t5)


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Summarized Text:
Сводка : Your input text here Wikipedia. Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia Wikipedia


**Keyframe selection**

In [ ]:
pip install torch torchvision transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
pip install av


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 16.2 MB/s eta 0:00:00


In [ ]:
import torch
import cv2
from torchvision.transforms import Compose, Resize, CenterCrop, Normalize
from transformers import CLIPModel

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# Define transformations for video frames
transform = Compose([
    Resize(224),
    CenterCrop(224),
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
])

def extract_keyframes(video_path, num_keyframes=5):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video file: {video_path}")

    # Extract visual features for each frame
    visual_features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = transform(frame)
        frame = torch.from_numpy(frame).unsqueeze(0).to(device)  # Convert to tensor
        with torch.no_grad():
            visual_feature = model.get_image_features(pixel_values=frame)
        visual_features.append(visual_feature)

    cap.release()

    visual_features = torch.cat(visual_features, dim=0)

    # Compute similarity scores between visual features
    similarity_scores = (visual_features @ visual_features.T).squeeze()

    # Select top keyframes based on similarity scores
    top_indices = torch.argsort(similarity_scores, descending=True)[:num_keyframes]
    keyframes = []

    cap = cv2.VideoCapture(video_path)
    for idx in top_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            keyframes.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    cap.release()

    return keyframes

# Example usage
video_path = "video_en.mp4"
num_keyframes = 25

keyframes = extract_keyframes(video_path, num_keyframes)
print("Selected Keyframes:", len(keyframes))


TypeError: Unexpected type <class 'numpy.ndarray'>